In [17]:
!pip install sentence_transformers
!pip install pytrec_eval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [28]:
from create_faiss_index import create_and_store_dense_index_approx
import pandas as pd
from run_query_faiss import query_dense_index
from sentence_transformers import SentenceTransformer
from dataloader import get_data
from tqdm import tqdm
import pytrec_eval
import os
import json

In [33]:
# # Example Usage
# # documents = [
# #     "The quick brown fox jumps over the lazy dog.",
# #     "Sentence embeddings are useful for dense retrieval.",
# #     "LaBSE is a multilingual model for sentence embeddings."
# # ]

tasks = ["monolingual","crosslingual"]
splits = ["train", "dev"]

model_name = "sentence-transformers/msmarco-bert-co-condensor"
model_name_short = "bert_codensor"

create_and_store_index = True



TASK = "monolingual"
SPLIT = "train"
# Get documents as a list of fact-checks from docs_monolingual.csv
doc_csv_path = f"docs_{TASK}_{SPLIT}.csv"
query_csv_path = f"queries_{TASK}_{SPLIT}.csv"
df_documents = pd.read_csv(doc_csv_path)
documents = df_documents['doc'].astype(str).tolist()

output_directory = f"{TASK}_{SPLIT}_dense_index_bert_codensor_ivf"
FINAL_OUTPUT_DIR = f'{model_name_short}_{TASK}_{SPLIT}'

device = "cuda"

# Index path 
index_path = f"{output_directory}/faiss_index_ivf"
metadata_path = f"{output_directory}/metadata.pkl"

## save results
if not os.path.exists(FINAL_OUTPUT_DIR):
    os.makedirs(FINAL_OUTPUT_DIR)


In [34]:
create_and_store_dense_index_approx(documents, model_name, output_directory)


Encoding documents...


Batches:   0%|          | 0/2680 [00:00<?, ?it/s]

Creating FAISS IndexIVFFlat...
Training FAISS index...
Saving FAISS index to monolingual_train_dense_index_bert_codensor_ivf/faiss_index_ivf...
Saving metadata to monolingual_train_dense_index_bert_codensor_ivf/metadata.pkl...
Index and metadata successfully saved.


In [35]:

# Load the model
model = SentenceTransformer(model_name)
model = model.to(device)
    


# Document csv
df_docs = pd.read_csv(doc_csv_path)
doc_ids = df_docs['doc_ids'].tolist()

# Query csv 
df_queries = pd.read_csv(query_csv_path)
query_ids = df_queries['query_ids'].tolist()
 
# df_fact_check_post_mapping
_, _, df_fact_check_post_mapping = get_data('./data')





In [36]:

# Metrics
metrics=['P_3', 'P_5', 'P_10', 'map_cut_3', 'map_cut_5', 'map_cut_10', 'ndcg_cut_3', 'ndcg_cut_5', 'ndcg_cut_10']
average_scores = {metric: 0.0 for metric in metrics}




count = 0

for _, row in tqdm(df_queries.iterrows(), total=len(df_queries)):
        
        qrels = {}
        runs = {}
        
        query = row['query']
        idx = query_ids[count] 
        
        #print("Query:", query)
        #print("Query Id:", idx)
        
        # Results from dense multilingual model 
        results = query_dense_index(query, model, index_path, metadata_path, top_k=10)

        ranked_fact_checks = []
        ranked_fact_checks_scores = []
        
        for rank, (doc, score) in enumerate(results, start=1):
            
            doc_id = doc_ids[doc]
            #print("Doc_id", doc_id)
            ranked_fact_checks.append(doc_id) 
            ranked_fact_checks_scores.append(score)
            
            #runs[str(idx)][str(doc_id)] = score 
            #print(f"Rank {rank}: {doc} (Score: {score})")
        
        
        runs[str(idx)] = {}
        qrels[str(idx)] = {} 
        
        for fc_id, score in zip(ranked_fact_checks, ranked_fact_checks_scores):
            runs[str(idx)][str(fc_id)] = float(score) 
        
        
        ground_truth = df_fact_check_post_mapping[
            df_fact_check_post_mapping['post_id'] == idx
        ]['fact_check_id'].tolist()

        ## keys must be strings
        for fc_id in ground_truth:
            qrels[str(idx)][str(fc_id)] = 1
        
        
        #qrels = {str(k): {str(d): v for d, v in docs.items()} for k, docs in qrels.items()}
        #run = {str(k): {str(d): v for d, v in docs.items()} for k, docs in runs.items()}

        #print("Metrics",metrics)

        evaluator = pytrec_eval.RelevanceEvaluator(qrels, metrics)
        #print("Evaluator", evaluator)
        
        #print("Qrels:", qrels)
        #print("Qrels type", type(qrels))
        #print("Runs:", runs)
        #print("Runs type", type(runs))
        results_scores = evaluator.evaluate(runs)

        #print("Results", results_scores)

        for query_id in results_scores:
            for metric in metrics:
                average_scores[metric] += results_scores[query_id][metric]

        with open(f"{FINAL_OUTPUT_DIR}/{model_name_short}_{TASK}_intermediate_scores_{count}.json", 'w') as f1:
            json.dump(average_scores, f1, indent=1)

        count = count + 1 
        # if count == 5:
        #     break

        

for metric in average_scores:
    average_scores[metric] /= count

print("Average scores:")
print(average_scores)



with open(f"{FINAL_OUTPUT_DIR}/{model_name_short}_{TASK}_finalresults.json", 'w') as f:
    json.dump(average_scores, f, indent=1)



100%|██████████| 4351/4351 [05:30<00:00, 13.17it/s]

Average scores:
{'P_3': 0.14602007201409684, 'P_5': 0.11220409101355637, 'P_10': 0.06809928752010984, 'map_cut_3': 0.265325340279374, 'map_cut_5': 0.28960213488597836, 'map_cut_10': 0.30458131318106, 'ndcg_cut_3': 0.2985609952416856, 'ndcg_cut_5': 0.3368495670112682, 'ndcg_cut_10': 0.3678352780605308}


In [ ]:
# some new models to try: ce rerankers  -for later
bert_multilingual = "amberoad/bert-multilingual-passage-reranking-msmarco"

# few more models for retriever:
distilbert = "msmarco-distilbert-base-v4"
MiniLM = "msmarco-MiniLM-L-12-v3"